In [1]:
import requests
import pandas as pd
import time
import gdown
import os
import glob
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path_data = 'drive/My Drive/PIP 2025 /Données propres/'

In [30]:
# Charger les fichiers CSVs georisques mensuels et annuels
df_mensuel = pd.read_csv(path_data+'catastrophes_naturelles_mensuelles.csv',dtype={"code_insee": str})
df_annuel = pd.read_csv(path_data+'catastrophes_naturelles_annuelles.csv',dtype={"code_insee": str})

<ipython-input-30-c5e6e1f2d15a>:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mensuel = pd.read_csv(path_data+'catastrophes_naturelles_mensuelles.csv',dtype={"code_insee": str})
<ipython-input-30-c5e6e1f2d15a>:3: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_annuel = pd.read_csv(path_data+'catastrophes_naturelles_annuelles.csv',dtype={"code_insee": str})


# Fichier annuel

## Avec ajout du poids des incendies suivant la surface touchée

In [31]:
# Créer une colonne "Nb événements ajusté" selon la règle donnée
df_annuel["Nb événements ajusté"] = np.where(
    df_annuel["Type catastrophe"] == "Incendie",
    np.ceil(df_annuel["Surface"] / 300000),  # Division et arrondi supérieur
    1  # Sinon, chaque événement compte pour 1
)

In [32]:
# Regroupement par Département, Type de catastrophe et Année
df_annuel_dep = df_annuel.groupby(["Département", "Nom département", "Type catastrophe", "Année"], as_index=False).agg(
    {"Nb événements ajusté": "sum"}  # Somme des événements ajustés
)

In [33]:
# Renommer la colonne pour plus de clarté
df_annuel_dep.rename(columns={"Nb événements ajusté": "Nombre événements"}, inplace=True)

In [34]:
df_annuel_inc_dep = df_annuel_dep[df_annuel_dep['Type catastrophe'] == "Incendie"]
df_annuel_inc_dep["Nombre événements"].sum()

4527.0

In [35]:
df_annuel_inc_dep['Département'].unique()

array(['11', '12', '13', '15', '16', '17', '18', '19', '21', '23', '24',
       '26', '29', '2A', '2B', '30', '32', '33', '34', '37', '38', '4',
       '40', '41', '43', '46', '47', '48', '49', '5', '50', '53', '56',
       '6', '63', '64', '65', '66', '7', '70', '71', '73', '74', '79',
       '8', '81', '83', '84', '85', '87'], dtype=object)

## Sans ajout du poids des incendies suivant la surface touchée

In [36]:
# Regroupement par Département, Type de catastrophe et Année
df_annuel_dep_old = df_annuel.groupby(["Département", "Nom département", "Type catastrophe", "Année"], as_index=False).agg(
    {"Code INSEE": "count"}  # Nombre d'événements et surface totale
)

# Renommer la colonne "Code INSEE" qui compte le nombre d'événements
df_annuel_dep_old.rename(columns={"Code INSEE": "Nombre événements"}, inplace=True)

In [37]:
# Afficher un aperçu du DataFrame regroupé
df_annuel_dep_old

,Département,Nom département,Type catastrophe,Année,Nombre événements
0,50,Manche,Inondation,1982,8
1,50,Manche,Inondation,1983,11
2,50,Manche,Inondation,1984,102
3,50,Manche,Inondation,1986,1
4,50,Manche,Inondation,1987,5
...,...,...,...,...,...
5449,95,Val-d'Oise,Sécheresse,2006,9
5450,95,Val-d'Oise,Sécheresse,2017,9
5451,95,Val-d'Oise,Sécheresse,2018,31
5452,95,Val-d'Oise,Sécheresse,2020,41


In [38]:
df_annuel_inc_dep_old = df_annuel_dep_old[df_annuel_dep['Type catastrophe'] == "Incendie"]
df_annuel_inc_dep_old["Nombre événements"].sum()

4193

In [39]:
df_annuel_inc_dep_old[df_annuel_inc_dep_old['Département'] == '07']

,Département,Nom département,Type catastrophe,Année,Nombre événements


# Fichier mensuel

## Avec ajout du poids des incendies suivant la surface touchée

In [40]:
# Créer une colonne "Nb événements ajusté" selon la règle donnée
df_mensuel["Nb événements ajusté"] = np.where(
    df_mensuel["Type catastrophe"] == "Incendie",
    np.ceil(df_mensuel["Surface"] / 300000),  # Division et arrondi supérieur
    1  # Sinon, chaque événement compte pour 1
)

In [41]:
# Regroupement par Département, Type de catastrophe et Année
df_mensuel_dep = df_mensuel.groupby(["Département", "Nom département", "Type catastrophe", "Année", "Mois"], as_index=False).agg(
    {"Nb événements ajusté": "sum"}  # Somme des événements ajustés
)

In [42]:
# Renommer la colonne pour plus de clarté
df_mensuel_dep.rename(columns={"Nb événements ajusté": "Nombre événements"}, inplace=True)
df_mensuel_dep

,Département,Nom département,Type catastrophe,Année,Mois,Nombre événements
0,10,Aube,Inondation,1983,4,36.0
1,10,Aube,Inondation,1985,5,17.0
2,10,Aube,Inondation,1987,9,5.0
3,10,Aube,Inondation,1988,5,8.0
4,10,Aube,Inondation,1988,6,1.0
...,...,...,...,...,...,...
22309,84,Vaucluse,Incendie,2021,8,8.0
22310,84,Vaucluse,Incendie,2021,9,1.0
22311,84,Vaucluse,Incendie,2022,8,2.0
22312,85,Vendee,Incendie,2020,5,1.0


In [43]:
df_mensuel_inc_dep = df_mensuel_dep[df_mensuel_dep["Type catastrophe"] == "Incendie"]
df_mensuel_inc_dep["Nombre événements"].sum()

4527.0

## Sans ajout du poids des incendies suivant la surface touchée

In [44]:
# Regroupement par Département, Type de catastrophe et Année
df_mensuel_dep_old = df_mensuel.groupby(["Département", "Type catastrophe", "Année", "Mois"], as_index=False).agg(
    {"Code INSEE": "count"}  # Nombre d'événements et surface totale
)

# Renommer la colonne "Code INSEE" qui compte le nombre d'événements
df_mensuel_dep_old.rename(columns={"Code INSEE": "Nombre événements"}, inplace=True)

In [45]:
df_mensuel_inc_dep_old = df_mensuel_dep_old[df_mensuel_dep_old['Type catastrophe'] == "Incendie"]
df_mensuel_inc_dep_old["Nombre événements"].sum()

4193

# Exportation des fichiers

In [46]:
# Exporter le DataFrame incendies en CSV
df_mensuel_dep.to_csv(path_data+"catastrophes_naturelles_mensuelles_dep_incendies.csv", index=False, encoding='utf-8-sig')
df_annuel_dep.to_csv(path_data+"catastrophes_naturelles_annuelles_dep_incendies.csv", index=False, encoding='utf-8-sig')

print("Extraction terminée !")

Extraction terminée !


In [47]:
# Exporter le DataFrame en CSV
df_mensuel_dep_old.to_csv(path_data+"catastrophes_naturelles_mensuelles_dep.csv", index=False, encoding='utf-8-sig')
df_annuel_dep_old.to_csv(path_data+"catastrophes_naturelles_annuelles_dep.csv", index=False, encoding='utf-8-sig')

print("Extraction terminée !")

Extraction terminée !
